In [14]:
from pathlib import Path
import numpy as np
import pandas as pd
import csv
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from etl.sources.census.etl_utils import get_state_fips_codes
from utils import unzip_file_from_url, remove_all_from_dir

DATA_PATH = Path.cwd().parent / "data"
TMP_PATH = DATA_PATH / "tmp"
EJSCREEN_FTP_URL = (
    "https://gaftp.epa.gov/EJSCREEN/2020/EJSCREEN_2020_StatePctile.csv.zip"
)
EJSCREEN_CSV = TMP_PATH / "EJSCREEN_2020_StatePctile.csv"
CSV_PATH = DATA_PATH / "dataset" / "ejscreen_2020"
print(DATA_PATH)

/Users/lucas/Documents/usds/repos/justice40-tool/score/data


In [15]:
# download file from ejscreen ftp
unzip_file_from_url(EJSCREEN_FTP_URL, TMP_PATH, TMP_PATH)

2021-06-28 17:25:06,680 [utils       ] INFO     Downloading https://gaftp.epa.gov/EJSCREEN/2020/EJSCREEN_2020_StatePctile.csv.zip
/Users/lucas/.virtualenvs/scoring/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gaftp.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2021-06-28 17:25:35,985 [utils       ] INFO     Extracting /Users/lucas/Documents/usds/repos/justice40-tool/score/data/tmp/downloaded.zip


In [16]:
df = pd.read_csv(EJSCREEN_CSV, dtype={"ID": "string"}, low_memory=False)

In [17]:
# write nationwide csv
CSV_PATH.mkdir(parents=True, exist_ok=True)
df.to_csv(CSV_PATH / f"usa.csv", index=False)

In [18]:
# write per state csvs
for fips in get_state_fips_codes(DATA_PATH):
    print(f"Generating data{fips} csv")
    df1 = df[df.ID.str[:2] == fips]
    # we need to name the file data01.csv for ogr2ogr csv merge to work
    df1.to_csv(CSV_PATH / f"data{fips}.csv", index=False)

Generating data01 csv
Generating data02 csv
Generating data04 csv
Generating data05 csv
Generating data06 csv
Generating data08 csv
Generating data09 csv
Generating data10 csv
Generating data11 csv
Generating data12 csv
Generating data13 csv
Generating data15 csv
Generating data16 csv
Generating data17 csv
Generating data18 csv
Generating data19 csv
Generating data20 csv
Generating data21 csv
Generating data22 csv
Generating data23 csv
Generating data24 csv
Generating data25 csv
Generating data26 csv
Generating data27 csv
Generating data28 csv
Generating data29 csv
Generating data30 csv
Generating data31 csv
Generating data32 csv
Generating data33 csv
Generating data34 csv
Generating data35 csv
Generating data36 csv
Generating data37 csv
Generating data38 csv
Generating data39 csv
Generating data40 csv
Generating data41 csv
Generating data42 csv
Generating data44 csv
Generating data45 csv
Generating data46 csv
Generating data47 csv
Generating data48 csv
Generating data49 csv
Generating

In [19]:
# cleanup
remove_all_from_dir(TMP_PATH)

2021-06-28 17:26:24,052 [utils       ] INFO     Removing EJSCREEN_2020_StatePctile.csv
2021-06-28 17:26:24,054 [utils       ] INFO     Removing .DS_Store
2021-06-28 17:26:24,056 [utils       ] INFO     Removing lead_score
2021-06-28 17:26:24,057 [utils       ] INFO     Removing Comparison Output.csv
2021-06-28 17:26:24,058 [utils       ] INFO     Removing calenviroscreen.zip
2021-06-28 17:26:24,060 [utils       ] INFO     Removing census
2021-06-28 17:26:24,102 [utils       ] INFO     Removing housing_and_transportation_index
2021-06-28 17:26:24,103 [utils       ] INFO     Removing CalEnviroScreen_4.0_2021.csv
